# B92

Import needed modules

In [1]:
from sympy.physics.quantum import qapply
from sympy.physics.quantum.gate import HadamardGate
from sympy.physics.quantum.qubit import Qubit
from random import getrandbits

from util.measure_all import measure_all_oneshot
from util.util import zero

## Alice part


Alice generates N random classical bits

In [2]:
def get_alice_bits(size):
    bits = []
    for i in range(size):
        bits.append(getrandbits(1))
    return bits

N=32
alice_bits = get_alice_bits(N)
print (f"Alice random bits = {alice_bits}")

Alice random bits = [0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1]


Alice encodes the random bits she generated using  a non-orthogonal basis `{|→>, |↗>} = {zero, plus}`
'''

where

    + = {|→>,|↑>} = {[1,0]T, [0,1]T}
    X = {|↖>,|↗>} = {1/sqtr(2)[-1,1]T, 1/sqtr(2)[1,1]T}


so that

    state  +      X
    |0>    |→>   |↖>
    |1>    |↑>   |↗>


In [3]:
def b92_alice_encoding(alice_bits):
    """
    :param alice_bits: Alice bits
    :return alice_encoded: alice encoded qubits
    """

    # we use a non-orthogonal base
    # {|→>, |↗>} = {zero, plus}
    alice_encoded = [qapply(HadamardGate(0) * Qubit(0)) if b == 1 else zero for b in alice_bits]

    return alice_encoded

alice_encoded = b92_alice_encoding(alice_bits)
print(f"Alice encoded: {alice_encoded}")

Alice encoded: [|0>, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, |0>, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, |0>, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2]


## Bob part


Bob generates N classical random bits

In [4]:
def get_bob_bits(size):
    bits = []
    for i in range(size):
        bits.append(getrandbits(1))
    return bits

bob_bits = get_bob_bits(N)
print (f"Bob random bits = {bob_bits}")

Bob random bits = [0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0]


For each bit he choses `Z` as a basis if the bit is zero or `X` if it is one

In [5]:
def b92_get_bob_basis(bits):
    bob_basis = ['Z' if b == 0 else 'X' for b in bits]
    return bob_basis

bob_basis = b92_get_bob_basis(bob_bits)
print(f"Bob basis: {bob_basis}")

Bob basis: ['Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z']


Bob measures in the random basis he picked

In [6]:
def b92_bob_measure(bob_basis, alice_encoded):
    """
    :param func bob_bases: function Bob bases
    :param qubit list alice_encoded: alice encoded qubits
    return measures: bob measured bits
    """

    measures = []
    for q, bb in zip(alice_encoded, bob_basis):
        qq = q
        if bb == 'X':
            # base change
            qq = qapply(HadamardGate(0) * q)
        m = measure_all_oneshot(qq)
        measures.append(m)
    return measures


bob_measures = b92_bob_measure(bob_basis, alice_encoded)
print(f"Bob measures: {bob_measures}")


Bob measures: [|0>, |0>, |1>, |0>, |0>, |0>, |1>, |0>, |0>, |0>, |0>, |0>, |0>, |0>, |1>, |0>, |0>, |1>, |1>, |1>, |1>, |0>, |1>, |0>, |0>, |0>, |0>, |1>, |0>, |0>, |0>, |1>]


## Final Steps

| Alice Basis | Alice Encoded | Bob Bits | Bob Bases| Result | Bob Measure | Matching | 
| ------------| ------------- | -------- | -------- | ------- | ------- | -------- |
| 0 | &#124;0>| 0 | Z | &#124;0> with 100% probability |0| Y | 
| 1 | &#124;+>| 0 | Z | &#124;0> with 50% probability  |0| N |
| 1 | &#124;+>| 0 | Z | &#124;1> with 50% probability  |1| N |
| 0 | &#124;0>| 1 | X | &#124;+> with 50% probability  |0| **Y** |
| 0 | &#124;0>| 1 | X | &#124;-> with 50% probability  |1| N |
| 1 | &#124;+>| 1 | X | &#124;+> with 100% probability |0| **Y** |

From this table we can see get an important point: if `Bob Measure` is 1 the corresponding bits in `Alice Basis` and `Bob Bases` do not match; this is not always true when `Bob Measure` is 0

This is the key of the algorithm

We look for ones in Bob measures

In [7]:
def find_ones(bob_bits):
    """
    :param bob_bits: Bob bits
    :return list of indexes where bob_bits are one
    """
    indices = []
    for i in range(len(bob_bits)):
        if bob_bits[i] == Qubit(1):
            indices.append(i)
    return indices

bob_one_indices = find_ones(bob_measures)
print(f"bob_one_indices={bob_one_indices}")


bob_one_indices=[2, 6, 14, 17, 18, 19, 20, 22, 27, 31]


And finally we generate the key
For Alice it is `1-alice_bit` where bob_indexes are one
For Bob it is bob_bit where bob_indexes are one

In [8]:
def b92_generate_keys(alice_bits, bob_bits, bob_one_indices):
    alice_key = []
    for i in bob_one_indices:
        alice_key.append(1 - alice_bits[i])
    
    bob_key = []
    for i in bob_one_indices:
        bob_key.append(bob_bits[i])
    return alice_key, bob_key

alice_key, bob_key = b92_generate_keys(alice_bits, bob_bits, bob_one_indices)

print(f"bob key={bob_key}")
print(f"alice key={alice_key}")

bob key=[0, 1, 1, 1, 0, 0, 0, 0, 1, 0]
alice key=[0, 1, 1, 1, 0, 0, 0, 0, 1, 0]


Check that the keys match

In [9]:
assert alice_key == bob_key

## Tests

In [10]:
def test_0():
    alice_bits=[0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1]
    bob_bits=[1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

    bob_one_indices=[1, 2, 7, 12, 16, 23]
    alice_key, bob_key = b92_generate_keys(alice_bits, bob_bits, bob_one_indices)
    print(f"bob key={bob_key}")
    print(f"alice key={alice_key}")
    assert alice_key == bob_key
test_0()

bob key=[0, 0, 0, 0, 0, 1]
alice key=[0, 0, 0, 0, 0, 1]
